# Ashsing topics to Wikidata Items, and to the Wikipedia Articles corresponding to those items.

This notebook takes the data generated from 00PrepareWikidataClaimsForTopicClassification.ipynb

I run this script in separeted notebook, for using Python Kernel instead of pyspark one.

In [6]:
#install fasttex with the patch to solve memory leak problem https://github.com/facebookresearch/fastText/pull/844
!pip install -U git+https://github.com/facebookresearch/fastText@02c61efaa6d60d6bb17e6341b790fa199dfb8c83 t 
!pip install tqdm #this optional. tdqm is just progress bar

    100% |████████████████████████████████| 61kB 7.9MB/s eta 0:00:01


In [43]:
!pip install -U numpy
!pip install -U pandas

Requirement already up-to-date: numpy in /srv/home/dsaez/venv/lib/python3.5/site-packages
    100% |████████████████████████████████| 10.3MB 140kB/s eta 0:00:01
    100% |████████████████████████████████| 512kB 2.3MB/s eta 0:00:01
Requirement already up-to-date: numpy>=1.13.3 in /srv/home/dsaez/venv/lib/python3.5/site-packages (from pandas)
Requirement already up-to-date: python-dateutil>=2.6.1 in /srv/home/dsaez/venv/lib/python3.5/site-packages (from pandas)
Requirement already up-to-date: six>=1.5 in /srv/home/dsaez/venv/lib/python3.5/site-packages (from python-dateutil>=2.6.1->pandas)
  Found existing installation: pytz 2016.7
    Not uninstalling pytz at /usr/lib/python3/dist-packages, outside environment /home/dsaez/venv
  Found existing installation: pandas 0.19.2
    Not uninstalling pandas at /usr/lib/python3/dist-packages, outside environment /home/dsaez/venv


In [1]:
# Read the output generated by 00PrepareWikidataClaimsForTopicClassification.ipynb
import pandas as pd
import pickle
allClaimsPerItemPandas = pd.read_csv('claimsPerWikidaItemTopicInputAllWikidataItems.csv')

In [2]:
#Read the model 
import fasttext
import numpy as np

model = fasttext.load_model('wikidata-topic-model-api/models/model.bin')


In [5]:
## Function to assign topics, copied from https://github.com/geohci/wikidata-topic-model-api/blob/master/app.py
def getLabels(claims_str,threshold=0.5):
    # make prediction
        lbls, scores = model.predict(claims_str, k=-1)
        results = {l:s for l,s in zip(lbls, scores)}
        sorted_res = [(l.replace("__label__", ""), results[l]) for l in sorted(results, key=results.get, reverse=True)]
        above_threshold = [r for r in sorted_res if r[1] >= threshold]
        lbls_above_threshold = []
        if above_threshold:
            for res in above_threshold:
                if res[1] > 0.5:
                    lbls_above_threshold.append(res[0])
        return above_threshold

In [6]:
allClaimsPerItemPandas.shape

(19276625, 2)

In [7]:
import csv
from tqdm import tqdm # tdqm is a progress bar
import gc

    
with open('topicsForAllWikidataItems.csv', 'w', newline='', encoding='utf-8') as csv_file:
    headers = ['Qid','topic','probability']
    writer = csv.writer(csv_file)
    writer.writerow(headers)
    for Q, BoW in tqdm(zip(allClaimsPerItemPandas.subject,allClaimsPerItemPandas.BoW)):
        topics = getLabels(BoW)
        for topic in topics:
              writer.writerow([Q,topic[0],round(topic[1],2)])



19276625it [1:13:41, 4359.28it/s]
